In [11]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
VERSION = 1

In [2]:
X_test = pd.read_csv('data/test.csv')
data_train = pd.read_csv('data/train.csv')
X = data_train.drop(columns=['label'])
y = data_train['label']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=20,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(100, activation="relu", input_shape=(784,)))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation ="softmax"))
model.compile(optimizer="adam", loss = 'sparse_categorical_crossentropy', metrics =['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])
model.save(f'assets/model_{VERSION}')

In [20]:
y_test = model.predict(X_test)
y_test = np.round(y_test)
y_test = pd.DataFrame(y_test)
y_test = y_test.idxmax(axis=1)

In [22]:
submission = pd.DataFrame(columns=['ImageId', 'Label'])
submission['ImageId'] = np.arange(1,28001)
submission['Label'] = y_test
submission.to_csv(f'submission_{VERSION}.csv', index=False)